# healthy_HCL_MCA_1_R_Seurat

+ 使用Seurat对HCL和MCA各个组织的数据进行单细胞标准流程的处理
    + 对于多样本的数据集，使用harmony去除批次效应
+ 输出存于`cache/healthy/Seurat_HCL_MCA`
    + 包括`ElbowPlot`,`Markers`,`obs`,`RDS`,`umap`

更新时间2024年4月5日

```bash

conda activate
cd ~/link/res_publish/run

jupyter nbconvert healthy_HCL_MCA_1_R_Seurat.ipynb --to python
mv healthy_HCL_MCA_1_R_Seurat.py healthy_HCL_MCA_1_R_Seurat.r
conda activate publish
nohup Rscript healthy_HCL_MCA_1_R_Seurat.r > nohup_healthy_HCL_MCA_1_R_Seurat.txt &
conda activate
jobs

```

In [1]:
source(file.path('~/link/res_publish','func_r_map_seurat.r'))
p_out = file.path(p_cache,'healthy/Seurat_HCL_MCA')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked


> function----------------------------------------
serurat_to_mtx

> Map_Seurat function-----------------------------
Map_Seurat_normalize
Map_Seurat_cluster
Map_Seurat_mapquery
precess_after_Seurat
run_Seurat			[simply]

> other-------------------------------------------
get_path_varmap


In [2]:
them_legend <- theme(
  legend.position = "inside",
  legend.justification = c(0, 0),
    rect = element_rect(fill = "transparent")
)

Map_Seruat_cluster_run_harmony =  function(
    adata,dims,
    resolution,key_batch,
    key_celltype=NULL,verbose=FALSE
){
    if(! key_batch %in% colnames(adata@meta.data)){
        stop(sprintf("[Error] key_batch = %s is not in adata@meta.data",key_batch))
    }

    adata@meta.data %>% count(.data[[key_batch]])
    adata <- adata %>% RunHarmony(key_batch, plot_convergence = TRUE, return.model = TRUE, verbose = verbose)
    DimPlot(object = adata, reduction = "pca", pt.size = .1, group.by = key_batch) + them_legend
    DimPlot(object = adata, reduction = "harmony", pt.size = .1, group.by = key_batch) + them_legend
    
    adata <- adata %>%
      RunUMAP(reduction = "harmony", dims = dims, verbose = verbose) %>%
      FindNeighbors(reduction = "harmony", dims = dims, verbose = verbose) %>%
      FindClusters(resolution = resolution, verbose = verbose)
    return(adata)
}

In [3]:
list_sp_db = list(
    human = 'HCL',
    mouse = 'MCA',
    HCL = 'human',
    MCA = 'mouse'
)
dims = 1:20
resolution = 0.1
verbose = FALSE

In [4]:
info <- read_csv(file.path(p_root, "run/cache", "info_healthy_HCL_MCA.csv"))
info <- info %>% mutate(
  db = map(info$sp, function(x) {
    return(list_sp_db[[x]])}),
    .before = tissue
)
q_tissue = str_split("Adrenal-Gland,Bone-Marrow,Brain,Heart,Intestine,Kidney,Liver,Lung,Spleen",',')[[1]]
info = info %>% filter(tissue %in% q_tissue)
info %>% head(2)

key_batch = 'X_batch'
# sp <- "human"
# tissue <- "Bone-Marrow"
# db = list_sp_db[[sp]]
# path <- file.path(p_root, "run/cache", sprintf("healthy/%s_%s", db, tissue))
# path
# dir.exists(path)

Rows: 22 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): tissue, sp, path, name, sp_simple

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


db,tissue,sp,path,name,sp_simple
<list>,<chr>,<chr>,<chr>,<chr>,<chr>
HCL,Adrenal-Gland,human,healthy/HCL_Adrenal-Gland,h_adr,h
MCA,Adrenal-Gland,mouse,healthy/MCA_Adrenal-Gland,m_adr,m


In [5]:
run  = function(db,tissue){
    if(file.exists(file.path(p_out,sprintf("RDS_%s_%s.rds", db, tissue)))){
        cat(sprintf("[has finish]%s_%s\n", db, tissue))
        return(0)
    }
    cat(sprintf("[start]%s_%s\n", db, tissue))

    key_batch = 'X_batch'
    
    options(repr.plot.height = 6, repr.plot.width = 6)
    adata = load_seuratobj(file.path(p_root, "run/cache", sprintf("healthy/%s_%s", db, tissue)))
    adata = Map_Seruat_normalize(adata)
    adata@meta.data %>% head(2)
    if(1 < adata@meta.data %>% count(.data[[key_batch]]) %>% nrow){
        
    adata = Map_Seruat_cluster_run_harmony(adata,dims,
        resolution,key_batch = key_batch)
    options(repr.plot.height = 6, repr.plot.width = 6 * 2)
    p1 = DimPlot(adata, reduction = "pca", label = FALSE, pt.size = .05, group.by =key_batch ) + them_legend
    p2 = DimPlot(adata, reduction = "harmony", label = FALSE, pt.size = .05, group.by = key_batch ) + them_legend
    file.path(p_out,sprintf("harmony_%s_%s.png", db, tissue)) %>% 
    ggsave(plot = (p1 + p2) ,dpi = 300,height = 1500,width = 1500*2,units = 'px',bg = 'transparent')
    
    }else{
        adata = Map_Seruat_cluster(adata,dims,resolution)
    options(repr.plot.height = 6, repr.plot.width = 6 )
    p1 = DimPlot(adata, reduction = "pca", label = FALSE, pt.size = .05, group.by =key_batch ) + them_legend
    file.path(p_out,sprintf("harmony_%s_%s.png", db, tissue)) %>% 
    ggsave(plot = p1 ,dpi = 300,height = 1500,width = 1500,units = 'px',bg = 'transparent')
    }
    
    df_umap = adata@reductions$umap@cell.embeddings %>% as.data.frame %>% rename(
        UMAP1 = umap_1,
        UMAP2 = umap_2)
    df_umap = df_umap %>% mutate(cell_name = rownames(df_umap))
    adata@meta.data = adata@meta.data %>% left_join(df_umap,by= c('cell_name' = 'cell_name'))
    rownames(adata@meta.data) = adata@meta.data$cell_name
    # saveRDS and obs
    adata@meta.data %>% write_csv(file.path(p_out,sprintf("obs_%s_%s.csv", db, tissue)))
    saveRDS(adata,file.path(p_out,sprintf("RDS_%s_%s.rds", db, tissue)))
    
    # plot
    ## ElbowPlot
    options(repr.plot.height = 6, repr.plot.width = 6)
    p = ElbowPlot(adata, ndims = 50)
    file.path(p_out,sprintf("ElbowPlot_%s_%s.png", db, tissue)) %>% 
    ggsave(dpi = 80,height = 200,width = 200,units = 'px',bg = 'transparent')
    
    ## UMAP pre_cell_type CL seurat_cluster
    options(repr.plot.height = 6, repr.plot.width = 6 * 3)
    p1 = DimPlot(adata, reduction = "umap", label = FALSE, pt.size = .05, group.by = key_batch) + them_legend
    p2 = DimPlot(adata, reduction = "umap", label = FALSE, pt.size = .05, group.by = "CL") + them_legend
    p3 = DimPlot(adata, reduction = "umap", label = FALSE, pt.size = .05, group.by = "seurat_clusters") + them_legend
    file.path(p_out,sprintf("umap_%s_%s.png", db, tissue)) %>% 
    ggsave(plot = (p1 + p2 + p3) ,dpi = 300,height = 1500,width = 1500*3,units = 'px',bg = 'transparent')
    
    # Markers
    markers <- FindAllMarkers(adata, only.pos = TRUE)
    markers  %>% write_csv(file.path(p_out,sprintf("Markers_%s_%s.csv", db, tissue)))
}

In [6]:
info %>% 
# filter( tissue == 'Bone-Marrow') %>% 
select(db,tissue) %>% pwalk(run)

[has finish]HCL_Adrenal-Gland
[has finish]MCA_Adrenal-Gland
[has finish]HCL_Bone-Marrow
[has finish]MCA_Bone-Marrow
[has finish]HCL_Brain
[has finish]MCA_Brain
[has finish]HCL_Heart
[has finish]MCA_Heart
[has finish]HCL_Intestine
[has finish]MCA_Intestine
[has finish]HCL_Kidney
[has finish]MCA_Kidney
[has finish]HCL_Liver
[has finish]MCA_Liver
[has finish]HCL_Lung
[has finish]MCA_Lung
[has finish]HCL_Spleen
[has finish]MCA_Spleen


# plot

In [7]:
# plot umap
run_2 = function(db,tissue){
    if(file.exists(file.path(p_out,sprintf("umap_%s_%s.png", db, tissue)))){
        cat(sprintf("[has finished]%s_%s\n", db, tissue))
        return()
    }
    
    cat(sprintf("[start]%s_%s\n", db, tissue))
    adata = readRDS(file.path(p_out,sprintf("RDS_%s_%s.rds", db, tissue)))

    ## UMAP pre_cell_type CL seurat_cluster
    options(repr.plot.height = 6, repr.plot.width = 6 * 3)
    p1 = DimPlot(adata, reduction = "umap", label = FALSE, pt.size = .05, group.by = key_batch) + them_legend
    p2 = DimPlot(adata, reduction = "umap", label = FALSE, pt.size = .05, group.by = "CL") + them_legend
    p3 = DimPlot(adata, reduction = "umap", label = FALSE, pt.size = .05, group.by = "seurat_clusters") + them_legend
    file.path(p_out,sprintf("umap_%s_%s.png", db, tissue)) %>% 
    ggsave(plot = (p1 + p2 + p3) ,dpi = 300,height = 1500,width = 1500*3,units = 'px',bg = 'transparent')

    # options(repr.plot.height = 6, repr.plot.width = 6)
    # p = ElbowPlot(adata, ndims = 50)
    # file.path(p_out,sprintf("ElbowPlot_%s_%s.png", db, tissue)) %>% 
    # ggsave(dpi = 80,height = 200,width = 200,units = 'px',bg = 'transparent')
    
    # df_umap = adata@reductions$umap@cell.embeddings %>% as.data.frame %>% rename(
    #     UMAP1 = umap_1,
    #     UMAP2 = umap_2)
    # df_umap = df_umap %>% mutate(cell_name = rownames(df_umap))
    # adata@meta.data = adata@meta.data %>% left_join(df_umap,by= c('cell_name' = 'cell_name'))
    # rownames(adata@meta.data) = adata@meta.data$cell_name
    # # saveRDS and obs
    # adata@meta.data %>% write_csv(file.path(p_out,sprintf("obs_%s_%s.csv", db, tissue)))
    # saveRDS(adata,file.path(p_out,sprintf("RDS_%s_%s.rds", db, tissue)))
}

info %>% 
# filter( tissue == 'Bone-Marrow') %>% 
select(db,tissue) %>% pwalk(run_2)
cat("--------------------\n[finish]\n--------------------")

[has finished]HCL_Adrenal-Gland
[has finished]MCA_Adrenal-Gland
[has finished]HCL_Bone-Marrow
[has finished]MCA_Bone-Marrow
[has finished]HCL_Brain
[has finished]MCA_Brain
[has finished]HCL_Heart
[has finished]MCA_Heart
[has finished]HCL_Intestine
[has finished]MCA_Intestine
[has finished]HCL_Kidney
[has finished]MCA_Kidney
[has finished]HCL_Liver
[has finished]MCA_Liver
[has finished]HCL_Lung
[has finished]MCA_Lung
[has finished]HCL_Spleen
[has finished]MCA_Spleen
--------------------
[finish]
--------------------

In [8]:
cat("--------------------\n[finish]\n--------------------")

--------------------
[finish]
--------------------

# show plot

In [9]:
df = tibble(
    name = file.path(list.files(p_out))
)
df = df %>% mutate(
    item = str_extract(name,"^[^_]+"),
    db = str_extract(name,"(?<=_)[HCLMA]{3}(?=_)"),
    tissue = str_extract(name,"(?<=[HCLMA]{3}_)[\\w-]+(?=\\.)")
)
df = df %>% arrange(tissue,db,item)
df %>% head(2)
df %>% dim

unique(df$item)
unique(df$tissue)

i = 'umap'
cat(sprintf("\n# %s\n\n",i))
df %>% filter(item == i) %>% select(name, db, tissue) %>%  pwalk(function(name, db, tissue) {
  cat(sprintf("> %s_%s\n![%s_%s](cache/healthy/Seurat_HCL_MCA/%s)\n\n", db, tissue, db, tissue, name), sep = "")
})

i = 'harmony'
cat(sprintf("\n# %s\n\n",i))
df %>% filter(item == "harmony") %>% select(name, db, tissue) %>%  pwalk(function(name, db, tissue) {
  cat(sprintf("> %s_%s\n![%s_%s](cache/healthy/Seurat_HCL_MCA/%s)\n\n", db, tissue, db, tissue, name), sep = "")
})


i = 'ElbowPlot'
cat(sprintf("\n# %s\n\n",i))
df %>% filter(item == i) %>% select(name, db, tissue) %>%  pwalk(function(name, db, tissue) {
  cat(sprintf("> %s_%s\n![%s_%s](cache/healthy/Seurat_HCL_MCA/%s)\n\n", db, tissue, db, tissue, name), sep = "")
})


name,item,db,tissue
<chr>,<chr>,<chr>,<chr>
ElbowPlot_HCL_Adrenal-Gland.png,ElbowPlot,HCL,Adrenal-Gland
Markers_HCL_Adrenal-Gland.csv,Markers,HCL,Adrenal-Gland


[1] 117   4

[1] "ElbowPlot"  "Markers"    "RDS"        "harmony"    "obs"       
 [6] "umap"       "SeuratUMAP" "info"       "Marker"     "Map"       
[11] "Singler"    "pre"

[1] "Adrenal-Gland" "Bone-Marrow"   "Brain"         "Heart"        
 [5] "Intestine"     "Kidney"        "Liver"         "Lung"         
 [9] "MCA"           "Spleen"        NA


# umap

> HCL_Adrenal-Gland
![HCL_Adrenal-Gland](cache/healthy/Seurat_HCL_MCA/umap_HCL_Adrenal-Gland.png)

> MCA_Adrenal-Gland
![MCA_Adrenal-Gland](cache/healthy/Seurat_HCL_MCA/umap_MCA_Adrenal-Gland.png)

> HCL_Bone-Marrow
![HCL_Bone-Marrow](cache/healthy/Seurat_HCL_MCA/umap_HCL_Bone-Marrow.png)

> MCA_Bone-Marrow
![MCA_Bone-Marrow](cache/healthy/Seurat_HCL_MCA/umap_MCA_Bone-Marrow.png)

> HCL_Brain
![HCL_Brain](cache/healthy/Seurat_HCL_MCA/umap_HCL_Brain.png)

> MCA_Brain
![MCA_Brain](cache/healthy/Seurat_HCL_MCA/umap_MCA_Brain.png)

> HCL_Heart
![HCL_Heart](cache/healthy/Seurat_HCL_MCA/umap_HCL_Heart.png)

> MCA_Heart
![MCA_Heart](cache/healthy/Seurat_HCL_MCA/umap_MCA_Heart.png)

> HCL_Intestine
![HCL_Intestine](cache/healthy/Seurat_HCL_MCA/umap_HCL_Intestine.png)

> MCA_Intestine
![MCA_Intestine](cache/healthy/Seurat_HCL_MCA/umap_MCA_Intestine.png)

> HCL_Kidney
![HCL_Kidney](cache/healthy/Seurat_HCL_MCA/umap_HCL_Kidney.png)

> MCA_Kidney
![MCA_Kidney](cache/healthy/Seurat_HCL_MCA

# umap

> HCL_Adrenal-Gland
![HCL_Adrenal-Gland](cache/healthy/Seurat_HCL_MCA/umap_HCL_Adrenal-Gland.png)

> MCA_Adrenal-Gland
![MCA_Adrenal-Gland](cache/healthy/Seurat_HCL_MCA/umap_MCA_Adrenal-Gland.png)

> HCL_Bone-Marrow
![HCL_Bone-Marrow](cache/healthy/Seurat_HCL_MCA/umap_HCL_Bone-Marrow.png)

> MCA_Bone-Marrow
![MCA_Bone-Marrow](cache/healthy/Seurat_HCL_MCA/umap_MCA_Bone-Marrow.png)

> HCL_Brain
![HCL_Brain](cache/healthy/Seurat_HCL_MCA/umap_HCL_Brain.png)

> MCA_Brain
![MCA_Brain](cache/healthy/Seurat_HCL_MCA/umap_MCA_Brain.png)

> HCL_Heart
![HCL_Heart](cache/healthy/Seurat_HCL_MCA/umap_HCL_Heart.png)

> MCA_Heart
![MCA_Heart](cache/healthy/Seurat_HCL_MCA/umap_MCA_Heart.png)

> HCL_Intestine
![HCL_Intestine](cache/healthy/Seurat_HCL_MCA/umap_HCL_Intestine.png)

> MCA_Intestine
![MCA_Intestine](cache/healthy/Seurat_HCL_MCA/umap_MCA_Intestine.png)

> HCL_Kidney
![HCL_Kidney](cache/healthy/Seurat_HCL_MCA/umap_HCL_Kidney.png)

> MCA_Kidney
![MCA_Kidney](cache/healthy/Seurat_HCL_MCA/umap_MCA_Kidney.png)

> HCL_Liver
![HCL_Liver](cache/healthy/Seurat_HCL_MCA/umap_HCL_Liver.png)

> MCA_Liver
![MCA_Liver](cache/healthy/Seurat_HCL_MCA/umap_MCA_Liver.png)

> HCL_Lung
![HCL_Lung](cache/healthy/Seurat_HCL_MCA/umap_HCL_Lung.png)

> MCA_Lung
![MCA_Lung](cache/healthy/Seurat_HCL_MCA/umap_MCA_Lung.png)

> HCL_Spleen
![HCL_Spleen](cache/healthy/Seurat_HCL_MCA/umap_HCL_Spleen.png)

> MCA_Spleen
![MCA_Spleen](cache/healthy/Seurat_HCL_MCA/umap_MCA_Spleen.png)